In [ ]:
from line_profiler import LineProfiler

In [300]:
from dataclasses import dataclass
from typing import List, Tuple
import numpy as np
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
from scipy.stats import norm
import scipy.stats as st
from statistics import NormalDist
from scipy.stats import multivariate_normal as mvn
import time
from scipy import stats


@dataclass
class NormalDistribution:
    mean: float
    std: float
    
@dataclass
class Part:
    
    type: str
    sub_part_name: str
    sensor: str
    signals: List   # Signal is numpy array of (500,3) with [frequency, Z, X]

In [ ]:

def load_part_data(part_type: str) -> List[Part]:
    
    parts = []
    for part_dir in os.listdir(f'psig_matcher/data/{part_type}'):
        
        sensor = part_dir[1:]
        measurement_files = glob.glob(f'psig_matcher/data/{part_type}/{part_dir}/*.npy')
        measurements = [np.load(f) for f in measurement_files]
        parts.append(Part(part_type, part_dir, sensor, measurements))
    
    return parts

con_parts = load_part_data('CON')
#conlid_parts = load_part_data('CONLID') # Need to handle the damage files
lid_parts = load_part_data('LID')

In [ ]:
import dataclasses


def limit_deminsionality(parts: List[Part], frequeny_indexes: List[int]) -> List[Part]:
    """Use only a subset of the frequencies for the analysis. This effectivley transforms the 
    500 dimension multivariant distribution to a n-dimentional distribution where n is the
    length of the frequency_indexes.
    
    Further, this assumes use of the X axis"""
    
    return [
        dataclasses.replace(part, signals=[[signal[index][1] for index in frequeny_indexes] for signal in part.signals])
        for part in parts]

In [ ]:
def estimate_normal_dist(x: List[float], confidence: float) -> NormalDistribution:
    """Estimate the normal distribution for the given data.
    This is done using: https://handbook-5-1.cochrane.org/chapter_7/7_7_3_2_obtaining_standard_deviations_from_standard_errors_and.htm#:~:text=The%20standard%20deviation%20for%20each,should%20be%20replaced%20by%205.15.
    
    TODO (henry): I'm not sure this is correct.
    """
    
    # Use T distribution for small sample sizes
    if len(x) < 30:
        lower, upper = st.t.interval(confidence, len(x)-1, loc=np.mean(x), scale=st.sem(x))
        t_value = st.t.ppf(confidence, len(x)-1)
        std = np.sqrt(len(x))*(upper-lower)*t_value
    
    # Use normal distribution for larger sample sizes
    else:
        lower, upper = st.norm.interval(confidence, loc=np.mean(x), scale=st.sem(x))
        z_value = st.norm.ppf(confidence)
        std = np.sqrt(len(x))*(upper-lower)*z_value
    
    return NormalDistribution(np.mean(x, axis=0), std)

In [ ]:
def concrete_normal_dist(x: List[float]) -> NormalDistribution:
    return NormalDistribution(np.mean(x, axis=0), np.std(x, axis=0))

In [ ]:
def plot_single_distributions(pdfs: List[NormalDistribution], labels: List[str], title: str):
    """ pdfs is a list of tuples of (mean, std) for each distribution."""
    
    for pdf, label in zip(pdfs, labels):
        
        x = np.linspace(pdf.mean - 3* pdf.std, pdf.mean + 3* pdf.std, 100)
        plt.plot(x, norm.pdf(x, pdf.mean, pdf.std), label=label)
    
    plt.legend(loc='best')
    plt.title(title)
    plt.show()

In [ ]:
def find_overlap(normal_d_1: NormalDistribution, normal_d_2: NormalDistribution) -> float:
    """Finds the overlap between two distributions."""
    
    return NormalDist(mu=normal_d_1.mean, sigma=normal_d_1.std).overlap(NormalDist(mu=normal_d_2.mean, sigma=normal_d_2.std))


In [ ]:
def find_overlap_of_set(pdfs: List[NormalDistribution]) -> float:
    """Finds the overlap between a set of distributions."""
    
    overlaps = []
    for i in range(len(pdfs)):
        for k in range(i+1, len(pdfs)):
            # Currently this method redundantly counts overlaps that may have already been accounted for. 
            # If A and B overlap on the edge of B, but C also overlaps on the edge of B, we're double
            # counting that overlap. Maybe we want to do this? 
            overlaps.append(find_overlap(pdfs[i], pdfs[k]))
    
    return np.mean(overlaps)

In [ ]:
def perform_1d_analysis(parts: List[Part]):
    
    single_d_parts = limit_deminsionality(parts, [0])
    print(single_d_parts[0].signals)
    pdfs = [estimate_normal_dist(part.signals, 0.95) for part in single_d_parts]
    plot_single_distributions(pdfs, [f"{part.type} - {part.sub_part_name}" for part in single_d_parts], f'1D Analysis - Estimated Confidence at 95%')
    print(f"Overlap of Estimated pdf's at 95% confidence: {find_overlap_of_set(pdfs)}")
    
    pdfs = [concrete_normal_dist(part.signals) for part in single_d_parts]
    plot_single_distributions(pdfs, [f"{part.type} - {part.sub_part_name}" for part in single_d_parts], f'1D Analysis - Concrete')
    print(f"Overlap of concrete pdf's: {find_overlap_of_set(pdfs)}")
    
    
    

perform_1d_analysis(con_parts)
#perform_1d_analysis(lid_parts)


In [233]:
def estimate_overlap_of_set_with_meta_pdf(pdfs: List[NormalDistribution], samples: int, confidence_bound: float) -> float:
    """Estimates the overlap between a set of distributions.
    
    The meta pdf is really just the combined pdfs of all the distributions, then we're drawing from that
    and seeing how many samples would cause conflicts. How can we prove the distribution we're pulling samples
    from is representative of the entire population? Is the estimated confidence good enough? 
    
    Could we potentially randomly sample from each distribution and just see which ones end up overlapping
    with the other distributions? TODO (henry): Think about this more 
    
    TODO (henry): This takes so long, like 5's per sample. We need to run 10,000's samples many times. 
    Need to computationall optimize this.
    """
    
    pdf_means = [pdf.mean for pdf in pdfs]    
    min_confidence = 1 - confidence_bound
    meta_pdf = estimate_normal_dist(pdf_means, 0.95)
    meta_samples = np.random.multivariate_normal(meta_pdf.mean, np.diag(meta_pdf.std), samples)
    mvn_pdfs = [mvn(mean=pdf.mean, cov=pdf.std) for pdf in pdfs]
        
    sample_confidences = [
        [pdf.cdf(sample) for pdf in mvn_pdfs]
        for sample in meta_samples]
    
    filtered_confidences = [
        list(filter(lambda confidence: confidence >= min_confidence, sample_confidence))
        for sample_confidence in sample_confidences]

    # We're ok with up to 1 match, but every one more than that is a conflict.
    collisions = [max(len(confidences)-1, 0) for confidences in filtered_confidences]
    return sum(collisions)/(samples*len(pdfs))
    

In [284]:
def estimate_overlap_of_set_with_individual_pdf(pdfs: List[NormalDistribution], samples: int, confidence_bound: float) -> float:
    """Estimates the overlap between a set of distributions.
    
    Pull from the samples from each individual distribution and see how many samples would cause conflicts 
    with other distributions. 
    
    Does this really represent the range of the entire population?
    
    TODO (henry): This takes so long, like 5's per sample per pdf. We need to run 10,000's samples many times. 
    Need to computationall optimize this.
    """
    
    min_confidence = 1 - confidence_bound
    mvn_pdfs = [mvn(mean=pdf.mean, cov=pdf.std) for pdf in pdfs]
    pdf_samples = [
        np.random.multivariate_normal(pdf.mean, np.diag(pdf.std), samples) 
        for pdf in pdfs]
    
    # flatten the samples
    pdf_samples = [sample for pdf_sample in pdf_samples for sample in pdf_sample]
    sample_confidences = [
        [pdf.cdf(sample) for pdf in mvn_pdfs]
        for sample in pdf_samples]
    
    filtered_confidences = [
        list(filter(lambda confidence: confidence >= min_confidence, sample_confidence))
        for sample_confidence in sample_confidences]
    
    # We're ok with up to 1 match, but every one more than that is a conflict.
    collisions = [max(len(confidences)-1, 0) for confidences in filtered_confidences]
    return sum(collisions)/(samples*len(pdfs))

In [301]:
def probability_of_multivariant_point(mu: List[float], cov: List[List[float]], x: List[float]) -> float:
    
    #https://stats.stackexchange.com/questions/331283/how-to-calculate-the-probability-of-a-data-point-belonging-to-a-multivariate-nor
    # Double check this math
    m_dist_x = np.dot((x-mu).transpose(),np.linalg.inv(cov))
    m_dist_x = np.dot(m_dist_x, (x-mu))
    return 1-stats.chi2.cdf(m_dist_x, 3)

In [315]:
def estimate_overlap_of_set_with_sample_signals(parts: List[Part], samples: int, confidence_bound: float) -> float:
    """ I believe this is the best solution out of all them. We are directly modeling the distribution/state space that
    the signals come from, and sampling from that. This directly correlates with the CI and is intuitive. See notion for
    more details and defense. """
    
    min_confidence = 1 - confidence_bound
    signals = [
        signal for part in parts 
        for signal in part.signals]
    
    part_pdfs = [estimate_normal_dist(part.signals, 0.95) for part in parts]
    #part_mvn_pdfs = [mvn(mean=pdf.mean, cov=pdf.std) for pdf in part_pdfs]
    sample_pdf = estimate_normal_dist(signals, 0.95)
    
    state_space_samples = np.random.multivariate_normal(sample_pdf.mean, np.diag(sample_pdf.std), samples)
    sample_confidences = [
        [probability_of_multivariant_point(pdf.mean, np.diag(pdf.std), sample) for pdf in part_pdfs]
        for sample in state_space_samples]

    filtered_confidences = [
        list(filter(lambda confidence: confidence >= min_confidence, sample_confidence))
        for sample_confidence in sample_confidences]

    # We're ok with up to 1 match, but every one more than that is a conflict.
    collisions = [max(len(confidences)-1, 0) for confidences in filtered_confidences]
    return sum(collisions)/(samples*len(part_pdfs))

In [295]:

def perform_multivariant_analsis(parts: List[Part]):
    
    # TOOD (henry): Abstract this out to run many times and plot CI until convergence.
    
    multivariant_parts = limit_deminsionality(parts, list(range(1)))
    pdfs = [estimate_normal_dist(part.signals, 0.95) for part in multivariant_parts]
    
    estimated_meta_collision_rate = estimate_overlap_of_set_with_meta_pdf(pdfs, 10, 0.95)
    print(f"Estimated collision rate from meta pdf: {estimated_meta_collision_rate}")
    
    estimated_set_collision_rate = estimate_overlap_of_set_with_individual_pdf(pdfs, 10, 0.95)
    print(f"Estimated collision rate from set of individual pdfs: {estimated_set_collision_rate}")
    
    estimated_sample_collision_rate = estimate_overlap_of_set_with_sample_signals(multivariant_parts, 10, 0.95)
    print(f"Estimated collision rate from sample distributiion: {estimated_sample_collision_rate}")

perform_multivariant_analsis(con_parts)

In [317]:
def run_meta_markov_multivariant_analysis(parts: List[Part]):
    
    multivariant_parts = limit_deminsionality(parts, list(range(1)))
    estimated_sample_collision_rate = estimate_overlap_of_set_with_sample_signals(multivariant_parts, 1000, 0.95)
    print(f"Estimated collision rate from sample distributiion: {estimated_sample_collision_rate}")
    

run_meta_markov_multivariant_analysis(con_parts)

sample_confidences: [[1.8673951274195133e-13, 0.0, 0.19558338804506203], [0.0, 0.0, 0.00011263819304796474], [2.6874634850848622e-08, 0.0, 0.8300003786984521], [0.0, 0.0, 9.222829744359728e-08], [0.0, 0.0, 2.0035591823286403e-05], [6.949654618448875e-08, 0.0, 0.8779368182142897], [2.8314110652738478e-06, 0.0, 0.9894833178696675], [8.72634138360251e-06, 0.0, 0.9983115449128294], [0.0, 0.0, 0.00013890689560647207], [4.049915958148631e-11, 0.0, 0.4372267631590303], [9.43689570931383e-15, 0.0, 0.11540869618644012], [0.0, 2.55351295663786e-15, 1.9066516143695367e-09], [0.0, 3.453358332627943e-08, 0.0], [0.000995474790494999, 0.0, 0.9516060805541245], [1.9584334154387761e-13, 0.0, 0.1971516327331324], [0.0, 0.0, 1.9563074857842366e-06], [4.440892098500626e-16, 0.0, 0.06295344844282535], [0.0, 1.5543122344752192e-15, 2.971342061286464e-09], [0.0, 0.0, 8.023851202132448e-06], [7.440979632455935e-08, 0.0, 0.8811309793939489], [1.1228786789274636e-09, 0.0, 0.6395317718197839], [0.0, 0.0, 0.02467